In [14]:
import requests

In [15]:
def get_json_for_state(state):

    url = f'https://static01.nyt.com/elections-assets/2020/data/api/2020-11-03/state-page/{state}.json'
    r = requests.get(url)
    if r.status_code == 200:
        return r.json()
    else:
        print(
            f"Something went wrong with state {state}. I'm outputing the raw response")
        return r

In [16]:
california_json = get_json_for_state('michigan')['data']

In [17]:
pres_race = list(filter(lambda x: x["race_name"] == "President", california_json['races']))[0]

In [18]:
pres_race.keys()

dict_keys(['race_id', 'race_slug', 'url', 'state_page_url', 'ap_polls_page', 'edison_exit_polls_page', 'race_type', 'election_type', 'election_date', 'runoff', 'race_name', 'office', 'officeid', 'nyt_race_description', 'race_rating', 'seat', 'seat_name', 'state_id', 'state_slug', 'state_name', 'state_nyt_abbrev', 'state_shape', 'party_id', 'uncontested', 'report', 'result', 'result_source', 'gain', 'lost_seat', 'votes', 'electoral_votes', 'absentee_votes', 'absentee_counties', 'absentee_count_progress', 'absentee_outstanding', 'absentee_max_ballots', 'provisional_outstanding', 'provisional_count_progress', 'poll_display', 'poll_countdown_display', 'poll_waiting_display', 'poll_time', 'poll_time_short', 'precincts_reporting', 'precincts_total', 'reporting_display', 'reporting_value', 'eevp', 'tot_exp_vote', 'eevp_source', 'eevp_value', 'eevp_display', 'county_data_source', 'incumbent_party', 'no_forecast', 'last_updated', 'candidates', 'has_incumbent', 'leader_margin_value', 'leader_mar

In [19]:
pres_timeseries = pres_race["timeseries"]
pres_timeseries[0]

{'vote_shares': {'trumpd': 0, 'bidenj': 0},
 'votes': 0,
 'eevp': 0,
 'eevp_source': 'edison',
 'timestamp': '2020-11-04T10:00:04Z'}

In [20]:
total_djt_diff = 0
total_jrb_diff = 0

In [21]:
def print_discrepancy(before_id):
    global total_djt_diff, total_jrb_diff
    
    before_update = pres_timeseries[before_id]
    after_update = pres_timeseries[before_id+1]
    
    votes_djt_before = round(before_update["vote_shares"]["trumpd"] * before_update["votes"])
    votes_jrb_before = round(before_update["vote_shares"]["bidenj"] * before_update["votes"])
    votes_djt_after = round(after_update["vote_shares"]["trumpd"] * after_update["votes"])
    votes_jrb_after = round(after_update["vote_shares"]["bidenj"] * after_update["votes"])

    diff_djt = votes_djt_after - votes_djt_before
    diff_jrb = votes_jrb_after - votes_jrb_before
    diff_total = after_update["votes"] - before_update["votes"]
    
    total_djt_diff += diff_djt
    total_jrb_diff += diff_jrb
    
    
    print("DISCREPANCY DETECTED:\n")
    print("DJT VOTES BEFORE: {}".format(votes_djt_before))
    print("JRB VOTES BEFORE: {}".format(votes_jrb_before))
    print("TOTAL BEFORE: {}".format(before_update["votes"]))
    print("TIMESTAMP BEFORE: {}\n".format(before_update["timestamp"]))

    print("DJT VOTES AFTER: {}".format(votes_djt_after))
    print("JRB VOTES AFTER: {}".format(votes_jrb_after))
    print("TOTAL AFTER: {}".format(after_update["votes"]))
    print("TIMESTAMP AFTER: {}\n".format(after_update["timestamp"]))


    print("DJT DIFF: {}".format(diff_djt))
    print("JRB DIFF: {}".format(diff_jrb))
    print("TOTAL DIFF: {}".format(diff_total))
    print("===================================")

In [22]:
for i,update in enumerate(pres_timeseries):
    before_update = pres_timeseries[i]
    after_update = pres_timeseries[i+1]
    
    # Calculate vote counts
    votes_djt_before = round(before_update["vote_shares"]["trumpd"] * before_update["votes"])
    votes_jrb_before = round(before_update["vote_shares"]["bidenj"] * before_update["votes"])
    votes_djt_after = round(after_update["vote_shares"]["trumpd"] * after_update["votes"])
    votes_jrb_after = round(after_update["vote_shares"]["bidenj"] * after_update["votes"])
    
    if((votes_djt_after < votes_djt_before) or (votes_jrb_after < votes_jrb_before)):
        print_discrepancy(i)
    
    if(i==len(pres_timeseries)-2):
        break
    
    

DISCREPANCY DETECTED:

DJT VOTES BEFORE: 133558
JRB VOTES BEFORE: 94536
TOTAL BEFORE: 232274
TIMESTAMP BEFORE: 2020-11-04T01:28:51Z

DJT VOTES AFTER: 133664
JRB VOTES AFTER: 94379
TOTAL AFTER: 232460
TIMESTAMP AFTER: 2020-11-04T01:32:32Z

DJT DIFF: 106
JRB DIFF: -157
TOTAL DIFF: 186
DISCREPANCY DETECTED:

DJT VOTES BEFORE: 163964
JRB VOTES BEFORE: 115572
TOTAL BEFORE: 284659
TIMESTAMP BEFORE: 2020-11-04T01:39:31Z

DJT VOTES AFTER: 164227
JRB VOTES AFTER: 115472
TOTAL AFTER: 285117
TIMESTAMP AFTER: 2020-11-04T01:39:54Z

DJT DIFF: 263
JRB DIFF: -100
TOTAL DIFF: 458
DISCREPANCY DETECTED:

DJT VOTES BEFORE: 331689
JRB VOTES BEFORE: 230117
TOTAL BEFORE: 573857
TIMESTAMP BEFORE: 2020-11-04T01:51:26Z

DJT VOTES AFTER: 326269
JRB VOTES AFTER: 233213
TOTAL AFTER: 574417
TIMESTAMP AFTER: 2020-11-04T01:51:52Z

DJT DIFF: -5420
JRB DIFF: 3096
TOTAL DIFF: 560
DISCREPANCY DETECTED:

DJT VOTES BEFORE: 540091
JRB VOTES BEFORE: 348240
TOTAL BEFORE: 909244
TIMESTAMP BEFORE: 2020-11-04T02:07:30Z

DJT VOTE

In [23]:
print("TOTAL DJT DIFF: {}".format(total_djt_diff))
print("TOTAL JRB DIFF: {}".format(total_jrb_diff))

TOTAL DJT DIFF: 17285
TOTAL JRB DIFF: 12173
